# Scrub
Handle any cleaning of data here. 

Issues that were discovered during EDA will be cleaned here.

In [1]:
import pandas as pd
import numpy as np

Read in the raw data and create a combined dataframe

In [2]:
# test_features_df = pd.read_csv('../data/raw/X_test.csv')
features_df = pd.read_csv('../data/raw/X_train.csv')
# features_df = test_features_df.append(train_features_df)
target_df = pd.read_csv('../data/raw/y_train.csv')
df = features_df.set_index('id').join(target_df.set_index('id')).reset_index()

## Columns

### Label encoding target column
Create column to hold numerical representation of the target column. This can be used for graphs during EDA, and is required for the modeling process.

In [3]:
df['status_group'] = df['status_group'].astype('category')
df['target'] = df['status_group'].cat.codes
df['target'].value_counts(normalize=False)

0    32259
2    22824
1     4317
Name: target, dtype: int64

In [4]:
df['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

### Dropping Columns

Not sure what 'num_private' is representing. Missing the description for it in the data dictionary that came with the data set.

In [5]:
to_drop = ['amount_tsh', 'date_recorded', 'id', 'wpt_name', 
           'num_private', 'region_code', 'lga', 
           'ward', 'public_meeting', 'recorded_by',
           'payment', 'extraction_type_group',
           'extraction_type_class', 'management_group',
           'quality_group', 'quantity_group', 'source_type',
           'source_class', 'waterpoint_type_group', 'status_group',
           'scheme_name', 'subvillage', 'funder', 'installer',
          ]

In [6]:
len(to_drop)

24

- **amount_tsh**: 70% of this column is 0's.
- **id**: Dropping Id because it contains all unique values.
- **date_recorded**: Is the date the row was entered, and was used to log when creating the dataset.
- **wpt_name**: The chosen name of the water well. Has no correlation to the functionality of the well.
- **num_private**: This column had no description with the dataset.
- **region_code**: Dropped because it indicates the same thing as region.
- **lga**: Geographic area.
- **ward**: Another geographic area. Dropping because there are numerous geographic area columns.
- **public_meeting**: Assumed this is an indicator for people gathering at the well, but it is unclear and there are a lot of missing values.
- **recorded_by**: The company who recorded the information. Is the same value for the entire dataset, so removing it.
- **payment**: Supposed to indicate the payment, but instead is just a copy of the payment_type.
- **extraction_type_group**: Most of these are copies of the extraction_group.
- **extraction_type_class**: Less specific version of extraction_type_group.
- **management_group**: Highly correlated with management.
- **quality_group**: Less specific version of quality.
- **quantity_group**: Duplicate of quantity column.
- **source_type**: Less specific version of source.
- **source_class**: Much less specific version of source.
- **waterpoint_type_group**: Less specific version of waterpoint_type.
- **status_group**: Categorical version of our target column.
- **scheme_name**: Has 36% missing values, and also is very similar to scheme_management
- **subvillage**: Has 10k unique values
- **funder**: High cardinality
- **installer**: High cardinality

In [7]:
df = df.drop(to_drop, axis=1)

## Duplicates

In [8]:
df = df.drop_duplicates()

In [9]:
df.duplicated().sum()

0

## Zeros or Missing Values
Two types of missing columns, columns that are missing less than 10%, and columns that are missing more than 30%.

Imputing:
- **funder**: Most commmon (mode) will add 3k rows funded by the gov't.
- **installer**: Most common (mode), DWE is the installer for the majority of the wells.
- **scheme_management**: Most common (mode), because one management company has majority of wells.
- **permit**: Drop missing values, because it is a boolean.
- **construction_year**: Dropping next three columns because they are all missing the same rows.
- **gps_height**: ''
- **population**: ''

### Loop through columns and change 'unknown' or 'other' to NaNs

In [10]:
df = df.replace(['unknown', 'other'], np.nan)

In [11]:
mode_impute = ['scheme_management']
to_drop = ['construction_year', 'gps_height', 'population', 'permit']

In [12]:
df = df[(df['construction_year'] > 0) & (df['population'] > 1) & (df['gps_height'] > 0)]

In [13]:
from sklearn.impute import SimpleImputer
mode_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [14]:
df[mode_impute] = mode_imputer.fit_transform(df[mode_impute])

In [15]:
df['permit'].value_counts()

True     19853
False     7840
Name: permit, dtype: int64

In [16]:
df = df.dropna()

In [17]:
df['target'].value_counts(normalize=False)

0    13412
2     5380
1     1579
Name: target, dtype: int64

## Outliers

## Feature Engineering

### Construction Year
TODO: create age column, and graph it

In [18]:
# check stats
print(df[df['construction_year'] != 0]['construction_year'].agg(['mean', 'median']))
df['construction_year'].value_counts().head(5)

mean      1998.233273
median    2002.000000
Name: construction_year, dtype: float64


2008    1662
2010    1615
2009    1571
2007     986
2006     910
Name: construction_year, dtype: int64

In [19]:
# if we cut out all the zeros, what does our target look like
df[(df['construction_year'] > 0)]['target'].value_counts()

0    13412
2     5380
1     1579
Name: target, dtype: int64

In [20]:
from datetime import datetime
# age = current_date - construction_year
current_date = datetime.now().year
df['age'] = current_date - df['construction_year']
df = df.drop(['construction_year'], axis=1)

### Save to CSV 

In [22]:
df.to_csv('../data/clean/tanzania.csv')